# Fuzzing with Constraints

In this chapter, we show how to extend grammars with _constraints_ – conditions that are evaluated while a string is produced, and which have to be satisfied.

**Prerequisites**

* You should have read the [chapter on efficient grammar fuzzing](GrammarFuzzer.ipynb).

## Specifying Constraints


In [ ]:
import fuzzingbook_utils

In [ ]:
from Grammars import EXPR_GRAMMAR, is_valid_grammar
from ProbabilisticGrammarFuzzer import opts, exp_opts, exp_string
from GrammarFuzzer import GrammarFuzzer, all_terminals
import copy

In [ ]:
constrained_expr_grammar = copy.deepcopy(EXPR_GRAMMAR)

constrained_expr_grammar.update(
    {
        "<start>": [("<expr>", opts(constraint="int($$) > 5"))]
    }
)

assert is_valid_grammar(constrained_expr_grammar)

In [ ]:
def exp_constraint(expansion):
    """Return the specified constraint, or None if unspecified"""
    if exp_opts(expansion) is None:
        return None
    return exp_opts(expansion).get('constraint', None)

In [ ]:
class SimpleConstraintGrammarFuzzer(GrammarFuzzer):
    pass

## Evaluating Constraints


In [ ]:
class SimpleConstraintGrammarFuzzer(SimpleConstraintGrammarFuzzer):
    def eval_constraint(self, tree, constraint):
        symbol, children = tree
        # print("Does", all_terminals(tree), "satisfy", constraint + "?")

        # Allow constraints to access the entire expression as $$
        all_rep = all_terminals(tree)
        expr = constraint.replace("$$", repr(all_rep))
        # print("$$ =", repr(all_rep))

        # Allow constraints to access the symbolic subexpressions as $10, $9, .., $1
        symbol_children = [c for c in children if len(c[1]) > 0]

        for i in range(len(symbol_children), 0, -1):
            child_rep = all_terminals(symbol_children[i - 1])
            # print("$" + repr(i), "=", repr(child_rep))
            expr = expr.replace("$" + repr(i), repr(child_rep))

        try:
            result = eval(expr)
        except Exception as exc:
            if self.log:
                print(constraint + ": " + expr + ": " + repr(exc))
            result = False

        # print(constraint, "=", result)
        return result

## Checking Constraints


In [ ]:
class SimpleConstraintGrammarFuzzer(SimpleConstraintGrammarFuzzer):
    # Return True iff all constraints of grammar are satisfied in TREE
    def constraints_satisfied(self, tree):
        symbol, children = tree
        if symbol not in self.grammar:
            return True

        applied_expansion = \
            "".join([child_symbol for child_symbol, _ in children])

        for expansion in self.grammar[symbol]:
            if exp_string(expansion) != applied_expansion:
                continue
            constraint = exp_constraint(expansion)
            if constraint is None:
                continue

            satisfied = self.eval_constraint(tree, constraint)
            if not satisfied:
                return False

        for c in children:
            if not self.constraints_satisfied(c):
                return False

        return True

The simplest method to check constraints is to retain only those trees that satisfy them.  Works, but can be very slow.

In [ ]:
class SimpleConstraintGrammarFuzzer(SimpleConstraintGrammarFuzzer):
    def fuzz_tree(self):
        while True:
            tree = super().fuzz_tree()
            if self.constraints_satisfied(tree):
                return tree

In [ ]:
constraint_grammar_fuzzer = SimpleConstraintGrammarFuzzer(constrained_expr_grammar)
expr = constraint_grammar_fuzzer.fuzz()
expr

In [ ]:
eval(expr)

In [ ]:
constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": [
         "+<factor>",
         "-<factor>",
         "(<expr>)",
         ("<integer>.<integer>", opts(constraint="int($$) > 10")),
         # ("<integer>.<integer>", opts(constraint=lambda: __ > 10)),
         ("<integer>", opts(constraint="int($$) > 10"))
        ],
    }
)

In [ ]:
from Timer import Timer

In [ ]:
constraint_grammar_fuzzer = SimpleConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Solving Constraints on the Go

Make things faster: Check as soon as (sub)tree is complete

In [ ]:
class FasterConstraintGrammarFuzzer(SimpleConstraintGrammarFuzzer):
    def expand_tree_once(self, tree):
        new_tree = super().expand_tree_once(tree)
        
        (symbol, children) = tree
        if all([exp_constraint(expansion) is None for expansion in self.grammar[symbol]]):
            # No constraints for this symbol
            return new_tree
                
        if self.any_possible_expansions(tree):
            # Still expanding
            return new_tree

        if self.constraints_satisfied(new_tree):
            # All constraints satisfied
            return new_tree

        # Replace tree by unexpanded symbol and try again
        if self.log:
            print(all_terminals(new_tree), "did not satisfy", symbol, "constraints")
            
        if self.replacement_attempts_counter > 0:
            if self.log:
                print("Trying another expansion")
            self.replacement_attempts_counter -= 1
            return (symbol, None)
        
        if self.log:
            print("Starting from scratch")
        raise RestartExpansionException

In [ ]:
class RestartExpansionException(Exception):
    pass

In [ ]:
class FasterConstraintGrammarFuzzer(FasterConstraintGrammarFuzzer):
    def __init__(self, grammar, replacement_attempts=10, **kwargs):
        super().__init__(grammar, **kwargs)
        self.replacement_attempts = replacement_attempts

    def fuzz_tree(self):
        while True:
            self.replacement_attempts_counter = self.replacement_attempts
            try:
                tree = super().fuzz_tree()
                return tree
            except RestartExpansionException:
                continue

In [ ]:
constraint_grammar_fuzzer = FasterConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Evaluation Shortcuts

\todo{Make things even faster: For `$n == x`, just replace `$n` by x}

In [ ]:
def exp_value(expansion):
    """Return the specified value generator, or None if unspecified"""
    if exp_opts(expansion) is None:
        return None
    return exp_opts(expansion).get('value', None)

In [ ]:
constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": [
         "+<factor>",
         "-<factor>",
         "(<expr>)",
         ("<integer>.<integer>", opts(value="{0: random.randint(100, 200)}")),
         ("<integer>", opts(value="random.randint(100, 200)")),

         # ("<integer>.<integer>", opts(fun=lambda: {1: random.randint(1000, 2000)})),
         # ("<integer>", opts(fun=lambda: random.randint(100, 200)))
        ],
    }
)

In [ ]:
class EvaluatingGrammarFuzzer(GrammarFuzzer):
    def expansion_to_children(self, expansion):
        children = super().expansion_to_children(expansion)
        value = exp_value(expansion)
        if value is None:
            return children
        
        result = eval(value)
        
        # print("Replacing", children, "by", repr(result))
        if isinstance(result, dict):
            for key in result:
                children[key] = (repr(result[key]), [])
        else:
            children = [(repr(result), [])]

        return children

In [ ]:
evaluating_fuzzer = EvaluatingGrammarFuzzer(constrained_expr_grammar)
evaluating_fuzzer.fuzz()

## All Together

In [ ]:
from ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer, ProbabilisticGrammarCoverageFuzzer

In [ ]:
class ConstraintGrammarFuzzer(ProbabilisticGrammarFuzzer,
                              FasterConstraintGrammarFuzzer,
                              EvaluatingGrammarFuzzer):
    pass

In [ ]:
class ConstraintGrammarCoverageFuzzer(ProbabilisticGrammarCoverageFuzzer, 
                                      FasterConstraintGrammarFuzzer,
                                      EvaluatingGrammarFuzzer):
    pass

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

1. Implement a syntax that allows people to refer to subtrees – say $1.$2 is the second child of the first symbol.


### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_